In [ ]:
import pandas as pd
import os
dir = os.getcwd()+'/work/csv'

#extract data from csv
def extract():
    try:
        # starting directory
        directory = dir
        # iterate over files in the directory
        for filename in os.listdir(directory):
            #get filename without ext
            file_wo_ext = os.path.splitext(filename)[0]
            # only process csv files
            if filename.endswith(".csv"):
                f = os.path.join(directory, filename)
                print(filename)
                # checking if it is a file
                if os.path.isfile(f):
                    df = pd.read_csv(f)
                    return df
    except Exception as e:
        #teams.send(to, "File Upload, Data extract error: ", f"Data extract error: File location {dir}" + str(e))
        print("Data extract error: " + str(e))

In [ ]:
df = extract()
print(df.info())

In [ ]:
print(df)

In [ ]:
#transform: 
# 1) eliminar matrículas duplicadas
# 2) resolver o que fazer com os valores nulos da materia 4 : trocar NaN por 0
# 3) inserir coluna descritiva sobre os alunos que falam ingles, ajustando valores nulos
# 4) eliminar nota ZERO de alunos sem reprovação (ainda não cursaram as matérias 1, 2, 3, 4)


In [ ]:
# 1) eliminar matrículas duplicadas

df[df['MATRICULA'].duplicated() == True]

In [ ]:
df = df.drop_duplicates(subset='MATRICULA', keep="first")
df

In [ ]:
print(df.info()) 

In [ ]:
# 2) resolver o que fazer com os valores nulos da materia 4 : trocar NaN por 0

df["NOTA_MAT_4"].fillna(0, inplace = True)

print(df.info()) 

In [ ]:
# 3) inserir coluna descritiva sobre os alunos que falam ingles, ajustando valores nulos

import numpy as np

df["INGLES"].fillna(-1, inplace = True)

conditions = [df['INGLES'] > 0, df['INGLES'] == 0, df['INGLES'] < 0]
choices = ['SIM', 'NÃO', 'SEM RESPOSTA']

df['INGLES_DESC'] = np.select(conditions, choices)

In [ ]:
print(df.info()) 


In [ ]:
print(df.groupby(['INGLES_DESC'])['INGLES_DESC'].count())

In [ ]:
# 4) eliminar nota ZERO de alunos sem reprovação (ainda não cursaram as matérias 1, 2, 3, 4)
# https://towardsdatascience.com/7-ways-to-handle-missing-values-in-machine-learning-1a6326adf79e

# Verificar se o aluno que tirou NOTA 4 foi aprovado ou reprovado nas matérias

df[(df['NOTA_MAT_1'] == 4)]

In [ ]:
# Verificar se o aluno cursou ou não a matéria 1

# APROVADO, NOTA >= 4
cond1_mat1 = (df['NOTA_MAT_1'] >= 4) & (df['REPROVACOES_MAT_1'] == 0)

# REPROVADO, NOTA < 4
cond2_mat1 = (df['NOTA_MAT_1'] < 4) & (df['REPROVACOES_MAT_1'] > 0)

# AINDA NAO CURSOU : NOTA = 0, SEM REPROVAÇÕES
cond3_mat1 = (df['NOTA_MAT_1'] == 0) & (df['REPROVACOES_MAT_1'] == 0)

conditions_MAT1 = [cond1_mat1, cond2_mat1, cond3_mat1]
choices = ['APROVADO', 'REPROVADO', 'AINDA NAO CURSOU']

df['CURSOU_MAT1_DESC'] = np.select(conditions, choices)

In [ ]:
print(df.groupby(['CURSOU_MAT1_DESC'])['CURSOU_MAT1_DESC'].count())

In [ ]:
# Verificar se o aluno cursou ou não as matérias 2, 3 e 4

# APROVADO, NOTA >= 4
cond1_mat2 = (df['NOTA_MAT_2'] >= 4) & (df['REPROVACOES_MAT_2'] == 0)
cond1_mat3 = (df['NOTA_MAT_3'] >= 4) & (df['REPROVACOES_MAT_3'] == 0)
cond1_mat4 = (df['NOTA_MAT_4'] >= 4) & (df['REPROVACOES_MAT_4'] == 0)

# REPROVADO, NOTA < 4
cond2_mat2 = (df['NOTA_MAT_2'] < 4) & (df['REPROVACOES_MAT_2'] > 0)
cond2_mat3 = (df['NOTA_MAT_3'] < 4) & (df['REPROVACOES_MAT_3'] > 0)
cond2_mat4 = (df['NOTA_MAT_4'] < 4) & (df['REPROVACOES_MAT_4'] > 0)

# AINDA NAO CURSOU : NOTA = 0, SEM REPROVAÇÕES
cond3_mat2 = (df['NOTA_MAT_2'] == 0) & (df['REPROVACOES_MAT_2'] == 0)
cond3_mat3 = (df['NOTA_MAT_3'] == 0) & (df['REPROVACOES_MAT_3'] == 0)
cond3_mat4 = (df['NOTA_MAT_4'] == 0) & (df['REPROVACOES_MAT_4'] == 0)

conditions_MAT2 = [cond1_mat2, cond2_mat2, cond3_mat2]
conditions_MAT3 = [cond1_mat3, cond2_mat3, cond3_mat3]
conditions_MAT4 = [cond1_mat4, cond2_mat4, cond3_mat4]
choices = ['APROVADO', 'REPROVADO', 'AINDA NAO CURSOU']

df['CURSOU_MAT2_DESC'] = np.select(conditions_MAT2, choices)
df['CURSOU_MAT3_DESC'] = np.select(conditions_MAT3, choices)
df['CURSOU_MAT4_DESC'] = np.select(conditions_MAT4, choices)

In [ ]:
print(df.groupby(['CURSOU_MAT2_DESC'])['CURSOU_MAT2_DESC'].count())
print('-')
print(df.groupby(['CURSOU_MAT3_DESC'])['CURSOU_MAT3_DESC'].count())
print('-')
print(df.groupby(['CURSOU_MAT4_DESC'])['CURSOU_MAT4_DESC'].count())

In [ ]:
#load data to dw
df.to_csv('curso_dw.csv', encoding='utf-8', index=False)